In [1]:
import pandas as pd
from environments.forex_tradning import tgym

C:\github\TradingGym\tvenv\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [2]:
parameters = {
    "observation_list":["open","high","low","close","minute","hour","weekday","RSI","SMA"],
    "stop_loss_max":200,
    "profit_taken_max":400    
}
file = "./data/split/GBPUSD/weekly/GBPUSD_20191110_W_148.csv"
file = "./data/split/GBPUSD/weekly/GBPUSD_20191208_W_1523.csvx"
file = "./data/split/GBPUSD/weekly/GBPUSD_20190915_W_1401.csvx"
df = pd.read_csv(file)
t = tgym(df, **parameters)

initial done:
observation_list:['open', 'high', 'low', 'close', 'minute', 'hour', 'weekday', 'RSI', 'SMA']
 assets:['GBPUSD' 'EURUSD']
 stop_loss_max:200
profit_taken_max: 400
time serial: 10/09/2019 0:00 -> 9/09/2019 9:55 length: 1440


In [3]:
import time
import datetime
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# if model: del model # remove to demonstrate saving and loading
model_name='./data/models/GBPUSD-week-20211021135753'
model = PPO.load(model_name)

start_time = time.time()
obs = t.reset()
t.current_step=0
done = False
while not done:
    action, _states = model.predict(obs)
    obs, rewards, done, info = t.step(action)
    t.render(mode='human')
print(f"--- running time: {(time.time() - start_time)}---")

  TicketSymbol  Type              ActionTime                             ActionPrice   MaxDD              CloseTimeClosePrice                                 Reward  SL      PT      DateDuration        Status  

       2 EURUSD      0 10/09/2019 0:05                  1.09992      -35 10/09/2019 11:10 1.10192                  190  200  200                   10        1
       4 EURUSD      0 10/09/2019 0:10                  1.09986      -29 10/09/2019 11:10 1.10186                  190  200  200                   10        1
       6 EURUSD      0 10/09/2019 0:15                  1.09991      -34 10/09/2019 11:10 1.10191                  190  200  200                   10        1
       8 EURUSD      0 10/09/2019 0:20                  1.09994      -37 10/09/2019 11:10 1.10194                  190  200  200                   10        1
      10 EURUSD      0 10/09/2019 0:25                  1.09996      -39 10/09/2019 11:10 1.10196                  190  200  200                   10   

In [ ]:
import time
import datetime
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# Parallel environments

start_time = time.time()
model_name = "./data/models/GBPUSD-week-" + datetime.datetime.now().strftime('%Y%m%d%H%M%S')
print(f'save model as {model_name}')
model = PPO("MlpPolicy", t, verbose=1)
model.learn(total_timesteps=250000)
model.save(model_name)
print(f"--- trainning time: {(time.time() - start_time)}---")


In [ ]:
0 > -22

In [ ]:
!python test.py >

In [ ]:
!script_output=$(python test.py 2>&1)
!echo $script_output